    collecting datasets from:
    https://www.kaggle.com/datasets/yewtsing/pretty-face?select=face
    https://www.kaggle.com/datasets/hereisburak/pins-face-recognition
    https://www.kaggle.com/datasets/ashwingupta3012/human-faces

**problem statement**:


In [1]:
import os
import hashlib
import numpy as np
import cv2
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torchvision import models, transforms
from torchvision.io import read_image 
from tqdm import tqdm
from sklearn.metrics import accuracy_score


**removing duplicates**

In [ ]:
def calculate_hash(image_path):
    try:
        with cv2.imread(image_path) as img:
            img = img.convert("RGB")
            hash_md5 = hashlib.md5(img.tobytes())
            return hash_md5.hexdigest()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def find_duplicates(folder_path):
    hashes = {}
    duplicates = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)

            if not file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
                continue

            img_hash = calculate_hash(file_path)
            if img_hash:
                if img_hash in hashes:
                    print(f"Duplicate found: {file_path} (same as {hashes[img_hash]})")
                    duplicates.append(file_path)
                else:
                    hashes[img_hash] = file_path

    for duplicate in duplicates:
        try:
            os.remove(duplicate)
            print(f"Deleted: {duplicate}")
        except Exception as e:
            print(f"Error deleting {duplicate}: {e}")

    print(f"Finished! Deleted {len(duplicates)} duplicate images.")

folder_path = r"dataset/"
for i in os.listdir(folder_path):
    find_duplicates(i)

**remove prefix**

In [ ]:
def remove_text_from_folders(parent_folder, text_to_remove):
    if not os.path.exists(parent_folder):
        print(f"Folder not found: {parent_folder}")
        return

    for folder_name in os.listdir(parent_folder):
        old_path = os.path.join(parent_folder, folder_name)

        if not os.path.isdir(old_path):
            continue

        if folder_name.startswith(text_to_remove):
            new_name = folder_name[len(text_to_remove):] 
            new_path = os.path.join(parent_folder, new_name)

            try:
                os.rename(old_path, new_path)
                print(f"Renamed: {folder_name} -> {new_name}")
            except Exception as e:
                print(f"Error renaming {folder_name}: {e}")

    print("Renaming complete!")

parent_folder = r"dataset/"
text_to_remove = "pins_" 
remove_text_from_folders(parent_folder, text_to_remove)

**croping and resizing the images to just the face**

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

input_folder = 'orginal_dataset'
output_folder = 'dataset'

os.makedirs(output_folder, exist_ok=True)

def crop_face(image_path, output_path):
    image = cv2.imread(image_path)
    faces = face_cascade.detectMultiScale(image, 1.1, 4)

    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        resized_face = cv2.resize(face, (224, 224))
        cv2.imwrite(output_path, resized_face)

for foldername in os.listdir(input_folder):
    for filename in os.listdir(input_folder+'/'+foldername):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, foldername +f'/{filename}')
            output_path = os.path.join(output_folder, foldername +f'/{filename}')
            
            crop_face(image_path, output_path)
            print(f"Processed {filename}")

print("Face cropping complete!")


Processed Adriana Lima0_0.jpg
Processed Adriana Lima101_3.jpg
Processed Adriana Lima102_4.jpg
Processed Adriana Lima103_5.jpg
Processed Adriana Lima104_6.jpg
Processed Adriana Lima105_7.jpg
Processed Adriana Lima106_8.jpg
Processed Adriana Lima107_9.jpg
Processed Adriana Lima108_10.jpg
Processed Adriana Lima109_11.jpg
Processed Adriana Lima10_2.jpg
Processed Adriana Lima110_13.jpg
Processed Adriana Lima111_14.jpg
Processed Adriana Lima112_15.jpg
Processed Adriana Lima113_16.jpg
Processed Adriana Lima114_17.jpg
Processed Adriana Lima115_18.jpg
Processed Adriana Lima116_19.jpg
Processed Adriana Lima118_20.jpg
Processed Adriana Lima11_12.jpg
Processed Adriana Lima120_22.jpg
Processed Adriana Lima121_23.jpg
Processed Adriana Lima122_24.jpg
Processed Adriana Lima123_25.jpg
Processed Adriana Lima124_26.jpg
Processed Adriana Lima125_27.jpg
Processed Adriana Lima126_28.jpg
Processed Adriana Lima127_29.jpg
Processed Adriana Lima128_30.jpg
Processed Adriana Lima129_31.jpg
Processed Adriana Lima1

**downsampling the images in the dataset to simplify further processes**

In [61]:
def preprocess_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        return print('error: no folder found')
    
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            img = cv2.imread(file_path)

            if img is None:
                print(f"Skipping non-image file: {file_name}")
                continue
            
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            resized_img = cv2.resize(rgb_img, (224, 224))

            r, g, b = cv2.split(resized_img)
            r_denoised = cv2.fastNlMeansDenoising(r, None, 10, 7, 21)
            g_denoised = cv2.fastNlMeansDenoising(g, None, 10, 7, 21)
            b_denoised = cv2.fastNlMeansDenoising(b, None, 10, 7, 21)
            denoised_img = cv2.merge([r_denoised, g_denoised, b_denoised])

            r, g, b = cv2.split(rgb_img)
            r_contrast = cv2.equalizeHist(r)
            g_contrast = cv2.equalizeHist(g)
            b_contrast = cv2.equalizeHist(b)
            contrast_img = cv2.merge([r_contrast, g_contrast, b_contrast])

            output_path = os.path.join(output_folder, file_name)
            cv2.imwrite(output_path, cv2.cvtColor(contrast_img, cv2.COLOR_RGB2BGR))
        print(f"Processed and saved: {file_path}")

folder_name_complex_list = ['Adriana Lima', 'Alex Lawther', 'Alexandra Daddario', 'Alvaro Morte', 'alycia dabnem carey', 'Amanda Crew', 'amber heard', 'Andy Samberg', 'Anne Hathaway', 'Anthony Mackie', 'Avril Lavigne', 'barack obama', 'barbara palvin', 'Ben Affleck', 'Bill Gates', 'Bobby Morley', 'Brenton Thwaites', 'Brian J. Smith', 'Brie Larson', 'camila mendes', 'Chris Evans', 'Chris Hemsworth', 'Chris Pratt', 'Christian Bale', 'Cristiano Ronaldo', 'Danielle Panabaker', 'Dominic Purcell', 'Dwayne Johnson', 'Eliza Taylor', 'Elizabeth Lail', 'elizabeth olsen', 'ellen page', 'elon musk', 'Emilia Clarke', 'Emma Stone', 'Emma Watson', 'gal gadot', 'grant gustin', 'Gwyneth Paltrow', 'Henry Cavil', 'Hugh Jackman', 'Inbar Lavi', 'Irina Shayk', 'Jake Mcdorman', 'Jason Momoa', 'jeff bezos', 'Jennifer Lawrence', 'Jeremy Renner', 'Jessica Barden', 'Jimmy Fallon', 'Johnny Depp', 'Josh Radnor', 'Katharine Mcphee', 'Katherine Langford', 'Keanu Reeves', 'kiernen shipka', 'Krysten Ritter', 'Leonardo DiCaprio', 'Lili Reinhart', 'Lindsey Morgan', 'Lionel Messi', 'Logan Lerman', 'Madelaine Petsch', 'Maisie Williams', 'margot robbie', 'Maria Pedraza', 'Marie Avgeropoulos', 'Mark Ruffalo', 'Mark Zuckerberg', 'Megan Fox', 'melissa fumero', 'Miley Cyrus', 'Millie Bobby Brown', 'Morena Baccarin', 'Morgan Freeman', 'Nadia Hilker', 'Natalie Dormer', 'Natalie Portman', 'Neil Patrick Harris', 'Pedro Alonso', 'Penn Badgley', 'Rami Malek', 'Rebecca Ferguson', 'Richard Harmon', 'Rihanna', 'Robert De Niro', 'Robert Downey Jr', 'Sarah Wayne Callies', 'scarlett johansson', 'Selena Gomez', 'Shakira Isabel Mebarak', 'Sophie Turner', 'Stephen Amell', 'Taylor Swift', 'Tom Cruise', 'tom ellis', 'Tom Hardy', 'Tom Hiddleston', 'Tom Holland', 'Tuppence Middleton', 'Ursula Corbero', 'Wentworth Miller', 'Zac Efron', 'Zendaya', 'Zoe Saldana']

for name in folder_name_complex_list:
    input_folder = rf"dataset\{name}"
    output_folder = rf"dataset\{name}"
    preprocess_images(input_folder, output_folder)

Processed and saved: dataset\Adriana Lima\Adriana Lima0_0.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima101_3.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima102_4.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima103_5.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima104_6.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima105_7.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima106_8.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima107_9.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima108_10.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima109_11.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima10_2.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima110_13.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima111_14.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima112_15.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima113_16.jpg
Processed and saved: dataset\Adriana 

**Appling image processing (edge detection)**

In [62]:
def preprocess_images2(input_folder, output_folder):
    if not os.path.exists(output_folder):
        return print('error: no folder found')
    
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            img = cv2.imread(file_path)

            if img is None:
                print(f"Skipping non-image file: {file_name}")
                continue
            
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            r, g, b = cv2.split(rgb_img)
            sobel_r_x = cv2.Sobel(r, cv2.CV_64F, 1, 0, ksize=3) 
            sobel_r_y = cv2.Sobel(r, cv2.CV_64F, 0, 1, ksize=3) 
            sobel_g_x = cv2.Sobel(g, cv2.CV_64F, 1, 0, ksize=3)
            sobel_g_y = cv2.Sobel(g, cv2.CV_64F, 0, 1, ksize=3)
            sobel_b_x = cv2.Sobel(b, cv2.CV_64F, 1, 0, ksize=3)
            sobel_b_y = cv2.Sobel(b, cv2.CV_64F, 0, 1, ksize=3)

            sobel_r = cv2.magnitude(sobel_r_x, sobel_r_y)
            sobel_g = cv2.magnitude(sobel_g_x, sobel_g_y)
            sobel_b = cv2.magnitude(sobel_b_x, sobel_b_y)

            sobel_img = cv2.merge([sobel_r, sobel_g, sobel_b])

            sobel_img = np.uint8(np.clip(sobel_img, 0, 255))

            output_path = os.path.join(output_folder, file_name)
            cv2.imwrite(output_path, sobel_img)
        print(f"Processed and saved: {file_path}")

folder_name_complex_list = ['Adriana Lima', 'Alex Lawther', 'Alexandra Daddario', 'Alvaro Morte', 'alycia dabnem carey', 'Amanda Crew', 'amber heard', 'Andy Samberg', 'Anne Hathaway', 'Anthony Mackie', 'Avril Lavigne', 'barack obama', 'barbara palvin', 'Ben Affleck', 'Bill Gates', 'Bobby Morley', 'Brenton Thwaites', 'Brian J. Smith', 'Brie Larson', 'camila mendes', 'Chris Evans', 'Chris Hemsworth', 'Chris Pratt', 'Christian Bale', 'Cristiano Ronaldo', 'Danielle Panabaker', 'Dominic Purcell', 'Dwayne Johnson', 'Eliza Taylor', 'Elizabeth Lail', 'elizabeth olsen', 'ellen page', 'elon musk', 'Emilia Clarke', 'Emma Stone', 'Emma Watson', 'gal gadot', 'grant gustin', 'Gwyneth Paltrow', 'Henry Cavil', 'Hugh Jackman', 'Inbar Lavi', 'Irina Shayk', 'Jake Mcdorman', 'Jason Momoa', 'jeff bezos', 'Jennifer Lawrence', 'Jeremy Renner', 'Jessica Barden', 'Jimmy Fallon', 'Johnny Depp', 'Josh Radnor', 'Katharine Mcphee', 'Katherine Langford', 'Keanu Reeves', 'kiernen shipka', 'Krysten Ritter', 'Leonardo DiCaprio', 'Lili Reinhart', 'Lindsey Morgan', 'Lionel Messi', 'Logan Lerman', 'Madelaine Petsch', 'Maisie Williams', 'margot robbie', 'Maria Pedraza', 'Marie Avgeropoulos', 'Mark Ruffalo', 'Mark Zuckerberg', 'Megan Fox', 'melissa fumero', 'Miley Cyrus', 'Millie Bobby Brown', 'Morena Baccarin', 'Morgan Freeman', 'Nadia Hilker', 'Natalie Dormer', 'Natalie Portman', 'Neil Patrick Harris', 'Pedro Alonso', 'Penn Badgley', 'Rami Malek', 'Rebecca Ferguson', 'Richard Harmon', 'Rihanna', 'Robert De Niro', 'Robert Downey Jr', 'Sarah Wayne Callies', 'scarlett johansson', 'Selena Gomez', 'Shakira Isabel Mebarak', 'Sophie Turner', 'Stephen Amell', 'Taylor Swift', 'Tom Cruise', 'tom ellis', 'Tom Hardy', 'Tom Hiddleston', 'Tom Holland', 'Tuppence Middleton', 'Ursula Corbero', 'Wentworth Miller', 'Zac Efron', 'Zendaya', 'Zoe Saldana']

for name in folder_name_complex_list:
    input_folder = rf"dataset\{name}"
    output_folder = rf"dataset_edge\{name}"
    preprocess_images2(input_folder, output_folder)

Processed and saved: dataset\Adriana Lima\Adriana Lima0_0.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima101_3.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima102_4.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima103_5.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima104_6.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima105_7.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima106_8.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima107_9.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima108_10.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima109_11.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima10_2.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima110_13.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima111_14.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima112_15.jpg
Processed and saved: dataset\Adriana Lima\Adriana Lima113_16.jpg
Processed and saved: dataset\Adriana 

**Fine-tuning model to classify specific celebrity**

**One short learning**

**resnet50 Network**

In [2]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.resnet50(pretrained=True)

        for param in self.resnet.parameters():
            param.requires_grad = False

        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        in_features = self.resnet.fc.in_features  
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.authorized_embeddings = {}

    def forward_one(self, x):
        embedding = self.resnet(x)  
        return F.normalize(embedding, p=2, dim=1)

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2
    
    def contrastive_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = (1 - label) * torch.pow(euclidean_distance, 2)  + (label) * torch.pow(torch.clamp(margin - euclidean_distance, min=0.0), 2)
        return loss.mean()
    
    def cosine_similarity_loss(self, output1, output2, label):
        cos_sim = F.cosine_similarity(output1, output2)
        loss = (1 - label) * torch.pow(cos_sim, 2) + (label) * torch.pow(torch.clamp(1 - cos_sim, min=0.0), 2)
        return loss.mean()
    
    def binary_cross_entropy_loss(self, output1, output2, label):
        similarity = F.cosine_similarity(output1, output2)
        loss = F.binary_cross_entropy_with_logits(similarity, label.float())
        return loss.mean()
    
    def mse_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = (label) * torch.pow(euclidean_distance, 2) + (1 - label) * torch.pow(torch.clamp(margin - euclidean_distance, min=0.0), 2)
        return loss.mean()
    
    def hinge_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = label * torch.pow(euclidean_distance, 2) + (1 - label) * torch.clamp(margin - euclidean_distance, min=0.0)
        return loss.mean()

    def autherise_user(self,image,name):
        self.eval()
        with torch.no_grad():
            embedding= self.forward_one(image)
        self.authorized_embeddings[name] = embedding
    
    def verify_autherisation(self,image,threshold=0.5):
        self.eval()
        with torch.no_grad():
            embedding_new = self.forward_one(image)

        closest_distance = float('inf')
        closest_user = None
        for user, embedding in self.authorized_embeddings.items():
            distance = torch.pairwise_distance(embedding_new, embedding).item()
            if distance < closest_distance:
                closest_distance = distance
                closest_user = user
        if closest_distance < threshold:
            return closest_user,closest_distance
        else:
            return '', closest_distance

In [3]:
class SiameseDataset(Dataset):
    def __init__(self, image_folder, transform=None, img_size=(224, 224)):
        self.image_folder = image_folder
        self.transform = transform
        self.img_size = img_size  

        self.classes = os.listdir(image_folder)
        self.image_paths = {class_idx: [] for class_idx, _ in enumerate(self.classes)}

        for class_idx, class_name in enumerate(self.classes):
            class_folder = os.path.join(image_folder, class_name)
            for img_name in os.listdir(class_folder):
                if img_name.endswith(('.jpg', '.jpeg', '.png')):  
                    self.image_paths[class_idx].append(os.path.join(class_folder, img_name))

        self.class_indices = list(self.image_paths.keys())

        self.resize = transforms.Resize(self.img_size)

    def __len__(self):
        return sum(len(imgs) for imgs in self.image_paths.values())

    def __getitem__(self, idx):
        all_images = [(img, lbl) for lbl, imgs in self.image_paths.items() for img in imgs]
        img1_path, label1 = all_images[idx]

        is_positive = np.random.choice([True, False])

        if is_positive:
            img2_path = np.random.choice(self.image_paths[label1])
            label2 = label1
        else:
            negative_label = np.random.choice([lbl for lbl in self.class_indices if lbl != label1])
            img2_path = np.random.choice(self.image_paths[negative_label])
            label2 = negative_label

        img1 = read_image(img1_path).float() / 255.0
        img2 = read_image(img2_path).float() / 255.0

        img1 = self.resize(img1)
        img2 = self.resize(img2)

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        label = torch.tensor(1 if label1 == label2 else 0, dtype=torch.float32)

        return img1, img2, label

In [ ]:
def transform_image(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),  
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),  
    ])
    return transform(image)


dataset = SiameseDataset(image_folder="dataset", transform=None)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0) 

for img1, img2, label in dataloader:
    print(img1.shape, img2.shape, label.shape)
    break

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0 
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    
    for input1, input2, label in progress_bar:
        input1, input2, label = input1.to(device), input2.to(device), label.to(device)
        
        output1, output2 = model(input1, input2)
        loss = model.binary_cross_entropy_loss(output1, output2, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss / len(dataloader):.4f}")

print("Training complete!")

saving/loading the model

In [14]:
torch.save(model.state_dict(), 'resnet50.pkl')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork()
state_dict = torch.load("resnet50.pkl")
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()
print('')

c:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\noobj\AppData\Local\Temp\ipykernel_4812\2028528105.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SEC

**Testing the model**

**manual test**

    test images:
    "dataset2\Adriana Lima\Adriana Lima0_0.jpg", "dataset2\Adriana Lima\Adriana Lima48_169.jpg"
    "dataset2\Alexandra Daddario\Alexandra Daddario5_388.jpg" , "dataset2\Alexandra Daddario\Alexandra Daddario54_393.jpg"
    "dataset2\Alvaro Morte\Alvaro Morte11_157.jpg"
    "dataset2\alycia dabnem carey\alycia dabnem carey7_185.jpg", "dataset2\alycia dabnem carey\alycia dabnem carey18_73.jpg"
    "dataset2\Amanda Crew\Amanda Crew11_10.jpg" 
    "dataset2\amber heard\amber heard4_367.jpg" , "dataset2\amber heard\amber heard21_319.jpg"

In [46]:
model.eval()  

test_image_path = r"dataset2\Adriana Lima\Adriana Lima0_0.jpg"
test_image = cv2.imread(test_image_path)
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image = transform_image(test_image)
test_image = test_image.to(device)
model.autherise_user(test_image,'Adriana')

In [12]:
model.eval()  

test_image_path = r"dataset2\amber heard\amber heard21_319.jpg"
test_image = cv2.imread(test_image_path)
test_image = transform_image(test_image)
test_image = test_image.to(device)
output = model.verify_autherisation(test_image,10)
print(output)

('Adriana Lima', 1.0225214958190918)


**automated test**

In [6]:
class testDataset(Dataset):
    def __init__(self, root_dir, authorized_users, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.authorized_users = authorized_users
        self.data = []
        self.authorized_image_name = [['dummy_path','dummy_user']]

        for user in os.listdir(root_dir):
            user_dir = os.path.join(root_dir, user)
            if os.path.isdir(user_dir):
                for img_name in os.listdir(user_dir):
                    img_path = os.path.join(user_dir, img_name)
                    if user in authorized_users:
                        label = 1
                        if user not in [row[1] for row in self.authorized_image_name]:
                            image = cv2.imread(img_path)
                            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                            if self.transform:
                                image = self.transform(image)
                            self.authorized_image_name.append([image,user])
                    else:
                        label = 0
                    self.data.append((img_path, label))  
        self.authorized_image_name.pop(0)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, label
    
def evaluate_with_authorized_users(model, dataset, threshold=0.1):
    data = [(img, label) for img, label in dataset]

    model.eval()
    with torch.no_grad():
        for image, user in dataset.authorized_image_name:
            image = image.to(device) 
            model.autherise_user(image, user)

    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for image, true_label in tqdm(data, desc="Evaluating Dataset"):
            image = image.to(device)
            predicted_user, _ = model.verify_autherisation(image, threshold)
            true_labels.append(true_label)
            predicted_labels.append(1 if predicted_user else 0)

    accuracy = accuracy_score(true_labels, predicted_labels)
    return accuracy

In [7]:
dataset_path = "dataset"
authorized_users = ["Adriana Lima", "Alexandra Daddario"]

transform = lambda img: transform_image(img).to(device) 
test_dataset = testDataset(dataset_path, authorized_users, transform=transform)

accuracy= evaluate_with_authorized_users(model, test_dataset, threshold=0.5)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Evaluating Dataset: 100%|██████████| 17534/17534 [28:35<00:00, 10.22it/s]  

Model Accuracy: 75.27%


**resnet101 network**

In [2]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.resnet101(pretrained=True)

        for param in self.resnet.parameters():
            param.requires_grad = False

        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        in_features = self.resnet.fc.in_features  
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.authorized_embeddings = {}

    def forward_one(self, x):
        embedding = self.resnet(x)  
        return F.normalize(embedding, p=2, dim=1)

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2
    
    def contrastive_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = (1 - label) * torch.pow(euclidean_distance, 2)  + (label) * torch.pow(torch.clamp(margin - euclidean_distance, min=0.0), 2)
        return loss.mean()
    
    def cosine_similarity_loss(self, output1, output2, label):
        cos_sim = F.cosine_similarity(output1, output2)
        loss = (1 - label) * torch.pow(cos_sim, 2) + (label) * torch.pow(torch.clamp(1 - cos_sim, min=0.0), 2)
        return loss.mean()
    
    def binary_cross_entropy_loss(self, output1, output2, label):
        similarity = F.cosine_similarity(output1, output2)
        loss = F.binary_cross_entropy_with_logits(similarity, label.float())
        return loss.mean()
    
    def mse_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = (label) * torch.pow(euclidean_distance, 2) + (1 - label) * torch.pow(torch.clamp(margin - euclidean_distance, min=0.0), 2)
        return loss.mean()
    
    def hinge_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = label * torch.pow(euclidean_distance, 2) + (1 - label) * torch.clamp(margin - euclidean_distance, min=0.0)
        return loss.mean()

    def autherise_user(self,image,name):
        self.eval()
        with torch.no_grad():
            embedding= self.forward_one(image)
        self.authorized_embeddings[name] = embedding
    
    def verify_autherisation(self,image,threshold=0.5):
        self.eval()
        with torch.no_grad():
            embedding_new = self.forward_one(image)

        closest_distance = float('inf')
        closest_user = None
        for user, embedding in self.authorized_embeddings.items():
            distance = torch.pairwise_distance(embedding_new, embedding).item()
            if distance < closest_distance:
                closest_distance = distance
                closest_user = user
        if closest_distance < threshold:
            return closest_user,closest_distance
        else:
            return '', closest_distance

In [3]:
class SiameseDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        
        self.classes = os.listdir(image_folder)
        self.image_paths = []

        for class_idx, class_name in enumerate(self.classes):
            class_folder = os.path.join(image_folder, class_name)
            for img_name in os.listdir(class_folder):
                if img_name.endswith(('.jpg', '.jpeg', '.png')):
                    self.image_paths.append((os.path.join(class_folder, img_name), class_idx))
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img1_path, label1 = self.image_paths[idx]
        
        img1 = cv2.imread(img1_path)  
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) 
        
        is_positive = np.random.choice([True, False])  
        
        if is_positive:
            same_class_images = [img for img, lbl in self.image_paths if lbl == label1]
            img2_path = np.random.choice(same_class_images)
            label2 = label1
        else:
            different_class_images = [img for img, lbl in self.image_paths if lbl != label1]
            img2_path = np.random.choice(different_class_images)
            label2 = np.random.choice([lbl for lbl in range(len(self.classes)) if lbl != label1])
        
        img2 = cv2.imread(img2_path) 
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB) 
        
        if self.transform:
            img1 = self.transform(img1).squeeze(0)
            img2 = self.transform(img2).squeeze(0)
        
        label = torch.tensor(1 if label1 == label2 else 0)
        
        return img1, img2, label

In [4]:
def transform_image(image):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224))  
    ])
    return transform(image).unsqueeze(0)

dataset = SiameseDataset(image_folder='dataset', transform=transform_image)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

for image1, image2,label in dataloader:
    print("Batch of images shape:", image1.shape)
    print("Batch of images shape:", image2.shape)
    print("Batch of labels shape:", label)
    break

Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels shape: tensor([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 1, 0, 0, 1, 0])


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
num_epochs = 8

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0 
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    
    for input1, input2, label in progress_bar:
        input1, input2, label = input1.to(device), input2.to(device), label.to(device)
        
        output1, output2 = model(input1, input2)
        loss = model.binary_cross_entropy_loss(output1, output2, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss / len(dataloader):.4f}")

print("Training complete!")

Epoch 1/8: 100%|██████████| 548/548 [03:17<00:00,  2.77batch/s, loss=0.63] 


Epoch 1/8, Average Loss: 0.6712


Epoch 2/8: 100%|██████████| 548/548 [03:16<00:00,  2.79batch/s, loss=0.653]


Epoch 2/8, Average Loss: 0.6637


Epoch 3/8: 100%|██████████| 548/548 [03:16<00:00,  2.78batch/s, loss=0.624]


...

Epoch 6/8, Average Loss: 0.6562


Epoch 7/8: 100%|██████████| 548/548 [03:14<00:00,  2.82batch/s, loss=0.623]


Epoch 7/8, Average Loss: 0.6516


Epoch 8/8: 100%|██████████| 548/548 [03:14<00:00,  2.81batch/s, loss=0.641]

Epoch 8/8, Average Loss: 0.6516
Training complete!


saving/loading the model

In [7]:
torch.save(model.state_dict(), 'resnet101.pkl')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork()
state_dict = torch.load("resnet101.pkl")
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()
print('')

c:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\noobj\AppData\Local\Temp\ipykernel_4812\2028528105.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SEC

**Automated Testing**

In [8]:
class testDataset(Dataset):
    def __init__(self, root_dir, authorized_users, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.authorized_users = authorized_users
        self.data = []
        self.authorized_image_name = [['dummy_path','dummy_user']]

        for user in os.listdir(root_dir):
            user_dir = os.path.join(root_dir, user)
            if os.path.isdir(user_dir):
                for img_name in os.listdir(user_dir):
                    img_path = os.path.join(user_dir, img_name)
                    if user in authorized_users:
                        label = 1
                        if user not in [row[1] for row in self.authorized_image_name]:
                            image = cv2.imread(img_path)
                            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                            if self.transform:
                                image = self.transform(image)
                            self.authorized_image_name.append([image,user])
                    else:
                        label = 0
                    self.data.append((img_path, label))  
        self.authorized_image_name.pop(0)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, label
    
def evaluate_with_authorized_users(model, dataset, threshold=0.1):
    data = [(img, label) for img, label in dataset]

    model.eval()
    with torch.no_grad():
        for image, user in dataset.authorized_image_name:
            image = image.to(device) 
            model.autherise_user(image, user)

    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for image, true_label in tqdm(data, desc="Evaluating Dataset"):
            image = image.to(device)
            predicted_user, _ = model.verify_autherisation(image, threshold)
            true_labels.append(true_label)
            predicted_labels.append(1 if predicted_user else 0)

    accuracy = accuracy_score(true_labels, predicted_labels)
    return accuracy

In [9]:
dataset_path = "dataset"
authorized_users = ["Adriana Lima", "Alexandra Daddario"]

transform = lambda img: transform_image(img).to(device) 
test_dataset = testDataset(dataset_path, authorized_users, transform=transform)

accuracy= evaluate_with_authorized_users(model, test_dataset, threshold=0.5)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Evaluating Dataset: 100%|██████████| 17534/17534 [27:37<00:00, 10.58it/s]

Model Accuracy: 71.96%


VGG-16 network

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.resnet = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

        for param in self.resnet.features.parameters():
            param.requires_grad = False

        in_features = self.resnet.classifier[0].in_features 
        self.resnet.classifier = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
        )

        self.authorized_embeddings = {}

    def forward_one(self, x):
        x = self.resnet.features(x)  

        x = x.view(x.size(0), -1)

        embedding = self.resnet.classifier(x)
        return F.normalize(embedding, p=2, dim=1)

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2
        
    def cosine_similarity_loss(self, output1, output2, label):
        cos_sim = F.cosine_similarity(output1, output2)
        loss = (1 - label) * torch.pow(cos_sim, 2) + (label) * torch.pow(torch.clamp(1 - cos_sim, min=0.0), 2)
        return loss.mean()

    def autherise_user(self,image,name):
        self.eval()
        with torch.no_grad():
            embedding= self.forward_one(image)
        self.authorized_embeddings[name] = embedding
    
    def verify_autherisation(self,image,threshold=0.5):
        self.eval()
        with torch.no_grad():
            embedding_new = self.forward_one(image)

        closest_distance = float('inf')
        closest_user = None
        for user, embedding in self.authorized_embeddings.items():
            distance = torch.pairwise_distance(embedding_new, embedding).item()
            if distance < closest_distance:
                closest_distance = distance
                closest_user = user
        if closest_distance < threshold:
            return closest_user,closest_distance
        else:
            return '', closest_distance

In [ ]:
class SiameseDataset(Dataset):
    def __init__(self, image_folder, transform=None, img_size=(224, 224)):
        self.image_folder = image_folder
        self.transform = transform
        self.img_size = img_size  

        self.classes = os.listdir(image_folder)
        self.image_paths = {class_idx: [] for class_idx, _ in enumerate(self.classes)}

        for class_idx, class_name in enumerate(self.classes):
            class_folder = os.path.join(image_folder, class_name)
            for img_name in os.listdir(class_folder):
                if img_name.endswith(('.jpg', '.jpeg', '.png')):  
                    self.image_paths[class_idx].append(os.path.join(class_folder, img_name))

        self.class_indices = list(self.image_paths.keys())

        self.resize = transforms.Resize(self.img_size)

    def __len__(self):
        return sum(len(imgs) for imgs in self.image_paths.values())

    def __getitem__(self, idx):
        all_images = [(img, lbl) for lbl, imgs in self.image_paths.items() for img in imgs]
        img1_path, label1 = all_images[idx]

        is_positive = np.random.choice([True, False])

        if is_positive:
            img2_path = np.random.choice(self.image_paths[label1])
            label2 = label1
        else:
            negative_label = np.random.choice([lbl for lbl in self.class_indices if lbl != label1])
            img2_path = np.random.choice(self.image_paths[negative_label])
            label2 = negative_label

        img1 = read_image(img1_path).float() / 255.0
        img2 = read_image(img2_path).float() / 255.0

        img1 = self.resize(img1)
        img2 = self.resize(img2)

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        label = torch.tensor(1 if label1 == label2 else 0, dtype=torch.float32)

        return img1, img2, label

In [ ]:
def transform_image(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),  
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),  
    ])
    return transform(image)

dataset = SiameseDataset(image_folder="dataset", transform=None)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=16) 

for img1, img2, label in dataloader:
    print(img1.shape, img2.shape, label.shape)
    break

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0 
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    
    for input1, input2, label in progress_bar:
        input1, input2, label = input1.to(device), input2.to(device), label.to(device)
        
        output1, output2 = model(input1, input2)
        loss = model.cosine_similarity_loss(output1, output2, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss / len(dataloader):.4f}")

print("Training complete!")

saving

In [ ]:

torch.save(model.state_dict(), 'vgg16.pkl')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork()
state_dict = torch.load("vgg16.pkl")
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()
print('')

testing

In [ ]:
class testDataset(Dataset):
    def __init__(self, root_dir, authorized_users, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.authorized_users = authorized_users
        self.data = []
        self.authorized_image_name = [['dummy_path','dummy_user']]

        for user in os.listdir(root_dir):
            user_dir = os.path.join(root_dir, user)
            if os.path.isdir(user_dir):
                for img_name in os.listdir(user_dir):
                    img_path = os.path.join(user_dir, img_name)
                    if user in authorized_users:
                        label = 1
                        if user not in [row[1] for row in self.authorized_image_name]:
                            image = cv2.imread(img_path)
                            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                            if self.transform:
                                image = self.transform(image)
                            self.authorized_image_name.append([image,user])
                    else:
                        label = 0
                    self.data.append((img_path, label))  
        self.authorized_image_name.pop(0)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, label
    
def evaluate_with_authorized_users(model, dataset, threshold=0.1):
    data = [(img, label) for img, label in dataset]

    model.eval()
    with torch.no_grad():
        for image, user in dataset.authorized_image_name:
            image = image.to(device) 
            model.autherise_user(image, user)

    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for image, true_label in tqdm(data, desc="Evaluating Dataset"):
            image = image.to(device)
            predicted_user, _ = model.verify_autherisation(image, threshold)
            true_labels.append(true_label)
            predicted_labels.append(1 if predicted_user else 0)

    accuracy = accuracy_score(true_labels, predicted_labels)
    return accuracy

In [ ]:
dataset_path = "dataset_test"
authorized_users = ["Adriana Lima", "Alexandra Daddario"]

transform = lambda img: transform_image(img).to(device) 
test_dataset = testDataset(dataset_path, authorized_users, transform=transform)

accuracy= evaluate_with_authorized_users(model, test_dataset, threshold=0.5)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

efficientnet_b0

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.efficientnet_b0(pretrained=True)

        for param in self.resnet.parameters():
            param.requires_grad = False

        in_features = self.resnet.classifier[1].in_features
        self.resnet.classifier = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.authorized_embeddings = {}

    def forward_one(self, x):
        embedding = self.resnet(x)
        return F.normalize(embedding, p=2, dim=1)

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

    def cosine_similarity_loss(self, output1, output2, label):
        cos_sim = F.cosine_similarity(output1, output2)
        loss = (1 - label) * torch.pow(cos_sim, 2) + (label) * torch.pow(torch.clamp(1 - cos_sim, min=0.0), 2)
        return loss.mean()

    def authorize_user(self, image, name):
        self.eval()
        with torch.no_grad():
            embedding = self.forward_one(image)
        self.authorized_embeddings[name] = embedding

    def verify_authorization(self, image, threshold=0.5):
        self.eval()
        with torch.no_grad():
            embedding_new = self.forward_one(image)

        closest_distance = float('inf')
        closest_user = None
        for user, embedding in self.authorized_embeddings.items():
            distance = torch.pairwise_distance(embedding_new, embedding).item()
            if distance < closest_distance:
                closest_distance = distance
                closest_user = user
        if closest_distance < threshold:
            return closest_user, closest_distance
        else:
            return '', closest_distance

In [ ]:
class SiameseDataset(Dataset):
    def __init__(self, image_folder, transform=None, img_size=(224, 224)):
        self.image_folder = image_folder
        self.transform = transform
        self.img_size = img_size  

        self.classes = os.listdir(image_folder)
        self.image_paths = {class_idx: [] for class_idx, _ in enumerate(self.classes)}

        for class_idx, class_name in enumerate(self.classes):
            class_folder = os.path.join(image_folder, class_name)
            for img_name in os.listdir(class_folder):
                if img_name.endswith(('.jpg', '.jpeg', '.png')):  
                    self.image_paths[class_idx].append(os.path.join(class_folder, img_name))

        self.class_indices = list(self.image_paths.keys())

        self.resize = transforms.Resize(self.img_size)

    def __len__(self):
        return sum(len(imgs) for imgs in self.image_paths.values())

    def __getitem__(self, idx):
        all_images = [(img, lbl) for lbl, imgs in self.image_paths.items() for img in imgs]
        img1_path, label1 = all_images[idx]

        is_positive = np.random.choice([True, False])

        if is_positive:
            img2_path = np.random.choice(self.image_paths[label1])
            label2 = label1
        else:
            negative_label = np.random.choice([lbl for lbl in self.class_indices if lbl != label1])
            img2_path = np.random.choice(self.image_paths[negative_label])
            label2 = negative_label

        img1 = read_image(img1_path).float() / 255.0
        img2 = read_image(img2_path).float() / 255.0

        img1 = self.resize(img1)
        img2 = self.resize(img2)

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        label = torch.tensor(1 if label1 == label2 else 0, dtype=torch.float32)

        return img1, img2, label

In [ ]:
def transform_image(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),  
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),  
    ])
    return transform(image)

dataset = SiameseDataset(image_folder="dataset_train", transform=None)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=16) 

for img1, img2, label in dataloader:
    print(img1.shape, img2.shape, label.shape)
    break

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0 
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    
    for input1, input2, label in progress_bar:
        input1, input2, label = input1.to(device), input2.to(device), label.to(device)
        
        output1, output2 = model(input1, input2)
        loss = model.cosine_similarity_loss(output1, output2, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss / len(dataloader):.4f}")

print("Training complete!")

saving

In [ ]:
torch.save(model.state_dict(), 'efficientnet_b0.pkl')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork()
state_dict = torch.load("efficientnet_b0.pkl")
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()
print('')

testing

In [ ]:
class testDataset(Dataset):
    def __init__(self, root_dir, authorized_users, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.authorized_users = authorized_users
        self.data = []
        self.authorized_image_name = [['dummy_path','dummy_user']]

        for user in os.listdir(root_dir):
            user_dir = os.path.join(root_dir, user)
            if os.path.isdir(user_dir):
                for img_name in os.listdir(user_dir):
                    img_path = os.path.join(user_dir, img_name)
                    if user in authorized_users:
                        label = 1
                        if user not in [row[1] for row in self.authorized_image_name]:
                            image = cv2.imread(img_path)
                            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                            if self.transform:
                                image = self.transform(image)
                            self.authorized_image_name.append([image,user])
                    else:
                        label = 0
                    self.data.append((img_path, label))  
        self.authorized_image_name.pop(0)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, label
    
def evaluate_with_authorized_users(model, dataset, threshold=0.1):
    data = [(img, label) for img, label in dataset]

    model.eval()
    with torch.no_grad():
        for image, user in dataset.authorized_image_name:
            image = image.to(device) 
            model.authorize_user(image, user)

    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for image, true_label in tqdm(data, desc="Evaluating Dataset"):
            image = image.to(device)
            predicted_user, _ = model.verify_authorization(image, threshold)
            true_labels.append(true_label)
            predicted_labels.append(1 if predicted_user else 0)

    accuracy = accuracy_score(true_labels, predicted_labels)
    return accuracy


In [ ]:
dataset_path = "dataset_test"
authorized_users = ["Pedro Alonso", "Sarah Wayne Callies"]

transform = lambda img: transform_image(img).to(device) 
test_dataset = testDataset(dataset_path, authorized_users, transform=transform)

accuracy= evaluate_with_authorized_users(model, test_dataset, threshold=0.5)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

**convnext_large network**

In [2]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.convnext = models.convnext_large(pretrained=True)

        for param in self.convnext.parameters():
            param.requires_grad = False
        for param in self.convnext.features[-1].parameters():
            param.requires_grad = True
        
        in_features = self.convnext.classifier[2].in_features
        self.convnext.classifier = nn.Sequential(nn.Flatten(),
                                                nn.Linear(in_features, 1024),
                                                nn.BatchNorm1d(1024),
                                                nn.ReLU(),
                                                nn.Dropout(0.5),
                                                nn.Linear(1024, 512),
                                                nn.BatchNorm1d(512),
                                                nn.ReLU(),
                                                nn.Dropout(0.5))

        self.authorized_embeddings = {}

    def forward_one(self, x):
        embedding = self.convnext(x)
        return F.normalize(embedding, p=2, dim=1)

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

    def contrastive_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = (1 - label) * torch.pow(euclidean_distance, 2)  + (label) * torch.pow(torch.clamp(margin - euclidean_distance, min=0.0), 2)
        return loss.mean()
    
    def cosine_similarity_loss(self, output1, output2, label):
        cos_sim = F.cosine_similarity(output1, output2)
        loss = (1 - label) * torch.pow(cos_sim, 2) + (label) * torch.pow(torch.clamp(1 - cos_sim, min=0.0), 2)
        return loss.mean()
    
    def binary_cross_entropy_loss(self, output1, output2, label):
        similarity = F.cosine_similarity(output1, output2)
        loss = F.binary_cross_entropy_with_logits(similarity, label.float())
        return loss.mean()
    
    def mse_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = (label) * torch.pow(euclidean_distance, 2) + (1 - label) * torch.pow(torch.clamp(margin - euclidean_distance, min=0.0), 2)
        return loss.mean()
    
    def hinge_loss(self, output1, output2, label, margin=1.0):
        euclidean_distance = torch.pairwise_distance(output1, output2)
        loss = label * torch.pow(euclidean_distance, 2) + (1 - label) * torch.clamp(margin - euclidean_distance, min=0.0)
        return loss.mean()

    def autherise_user(self,image,name):
        self.eval()
        with torch.no_grad():
            embedding= self.forward_one(image)
        self.authorized_embeddings[name] = embedding
    
    def verify_autherisation(self,image,threshold=0.5):
        self.eval()
        with torch.no_grad():
            embedding_new = self.forward_one(image)

        closest_distance = float('inf')
        closest_user = None
        for user, embedding in self.authorized_embeddings.items():
            distance = torch.pairwise_distance(embedding_new, embedding).item()
            if distance < closest_distance:
                closest_distance = distance
                closest_user = user
        if closest_distance < threshold:
            return closest_user,closest_distance
        else:
            return '', closest_distance
        

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
num_epochs = 8

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0 
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    
    for input1, input2, label in progress_bar:
        input1, input2, label = input1.to(device), input2.to(device), label.to(device)
        
        output1, output2 = model(input1, input2)
        loss = model.binary_cross_entropy_loss(output1, output2, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss / len(dataloader):.4f}")

print("Training complete!")

c:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Large_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/8: 100%|██████████| 548/548 [08:17<00:00,  1.10batch/s, loss=0.689]


Epoch 1/8, Average Loss: 0.6948


Epoch 2/8: 100%|██████████| 548/548 [07:59<00:00,  1.14batch/s, loss=0.697]


Epoch 2/8, Average Loss: 0.6788


Epoch 3/8: 100%|██████████| 548/548 [07:49<00:00,  1.17batch/s, loss=0.693]


...

Epoch 6/8, Average Loss: 0.6716


Epoch 7/8: 100%|██████████| 548/548 [07:48<00:00,  1.17batch/s, loss=0.662]


Epoch 7/8, Average Loss: 0.6722


Epoch 8/8: 100%|██████████| 548/548 [08:16<00:00,  1.10batch/s, loss=0.685]

Epoch 8/8, Average Loss: 0.6747
Training complete!


In [ ]:
dataset_path = "dataset"
authorized_users = ["Adriana Lima", "Alexandra Daddario"]

transform = lambda img: transform_image(img).to(device) 
test_dataset = testDataset(dataset_path, authorized_users, transform=transform)

accuracy= evaluate_with_authorized_users(model, test_dataset, threshold=0.5)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 51.65%


**using webcam**

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
camera = cv2.VideoCapture(0)

if not camera.isOpened():
    print("Error: Could not access the camera.")
    exit()
print("Press 'q' to quit the live stream.")
print("Press 'c' to capture the live stream.")

save_folder = 'autherised_user_images'
os.makedirs(save_folder, exist_ok=True)

while True:
    ret, frame = camera.read()
    
    if not ret:
        print("Error: Failed to capture frame.")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Live Camera Stream', frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('c') and len(faces) > 0:
        name = input("Enter your name: ")
        face_image = frame[y:y+h, x:x+w]
        if name:
            filename = os.path.join(save_folder, f"{name}.jpg")
        else:
            filename = os.path.join(save_folder, f"unknow{np.random.randint(100)}.jpg")
        cv2.imwrite(filename, face_image)
        print("Image saved as:", filename)

        print(f"Name: {name}")

    if key == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

Press 'q' to quit the live stream.
Press 'c' to capture the live stream.
Image saved as: autherised_user_images\Jewel.jpg
Name: Jewel
Image saved as: autherised_user_images\jewel2.jpg
Name: jewel2
